In [9]:
import tensorflow as tf

# Set Deterministic Training

In [10]:
import os
import numpy as np
import random
from glob import glob
from time import time

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Define Hyper-Parameters

In [11]:
PREPROCESSING_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': 0.016,
    'frame_step_in_s': 0.016,
    'lower_frequency': 20,
    'upper_frequency': 4000,
    'num_mel_bins': 40,
    'num_coefficients': 10
}
TRAINING_ARGS = {
    'batch_size': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 30
}

#Optimization parameter
alpha = 0.3
final_sparsity = 0.80

# Train/Val/Test datasets only with Stop/Go

In [12]:
train_ds = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
val_ds = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
test_ds = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])

## Preprocess - Extraction of MFCC's

In [13]:
from preprocessing import LABELS
from preprocessing import get_mfccs
from functools import partial

def get_mfccs_and_label(filename, downsampling_rate, frame_length_in_s, frame_step_in_s, num_mel_bins, lower_frequency, upper_frequency, num_coefficients):
    mfccs, label = get_mfccs(filename, downsampling_rate,
                            frame_length_in_s,
                            frame_step_in_s,
                            num_mel_bins,
                            lower_frequency,
                            upper_frequency,
                            num_coefficients
                            )
    
    return mfccs, label



#Parameter freezing
get_frozen_mfccs = partial(get_mfccs_and_label, **PREPROCESSING_ARGS)


In [14]:
def preprocess(filename):
    #Get mfccs and then expand the dimension to then pass it to the model
    processed_signal, label = get_frozen_mfccs(filename)
    processed_signal = tf.expand_dims(processed_signal, -1) #because we need a 3d tensor for the model input

    label_id = tf.argmax(label == LABELS)

    return processed_signal, label_id    

In [15]:
#Setting up batch size and number of epochs
batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

#Preprocess all the datasets
train_ds = train_ds.map(preprocess).batch(batch_size).cache()
val_ds = val_ds.map(preprocess).batch(batch_size)
test_ds = test_ds.map(preprocess).batch(batch_size)

2022-12-19 17:53:55.907307: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-12-19 17:53:55.907529: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 AVX512F FMA
2022-12-19 17:53:56.213652: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 17:53:56.215118: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 17:53:56.215281: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 17:53:56.559563: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 17:53:56.561110: W tensorflow/core/framework/op_kernel.cc:1780] O

# Create the Model

In [16]:
#Infos on what the input of the model
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (20, 62, 10, 1)
Data Shape: (62, 10, 1)
Labels: tf.Tensor([0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 0], shape=(20,), dtype=int64)
2022-12-19 17:53:59.297761: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [17]:
for example_batch, example_labels in train_ds.take(1):
    pass

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    
    # ----------------------- Normal convolution  ------------------------------------------
    tf.keras.layers.Conv2D(filters=int(256 * alpha), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.ReLU(),
    # ----------------------- Depth wise convolution 2 ------------------------------------------
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], 
        use_bias=False, padding='same'),
    
    tf.keras.layers.Conv2D(filters=int(256 * alpha), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.ReLU(),

    # ----------------------- Depth wise convolution 3 ------------------------------------------    
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    
    tf.keras.layers.Conv2D(filters=int(256 * alpha), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.GlobalAveragePooling2D(),
    
    tf.keras.layers.Dense(units=len(LABELS)),
    
    tf.keras.layers.Softmax()
])
print('filters are of dimension:', int(256 * alpha))

filters are of dimension: 76
2022-12-19 17:54:00.088052: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 4, 76)         684       
                                                                 
 batch_normalization (BatchN  (None, 30, 4, 76)        304       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 30, 4, 76)         0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 30, 4, 76)        684       
 Conv2D)                                                         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 4, 76)         5776      
                                                                 
 batch_normalization_1 (Batc  (None, 30, 4, 76)        3

## Setup Magnitude-based Weights Pruning

In [19]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epochs * 0.2)
end_step = int(len(train_ds) * epochs)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 30, 4, 76)        1370      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 30, 4, 76)        305       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 30, 4, 76)        1         
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_depthwi  (None, 30, 4, 76)        685       
 se_conv2d (PruneLowMagnitud                                     
 e)                                                     

# Train the Model

In [20]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

#Compiling and running the model
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)
history = model_for_pruning.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)

Epoch 1/30
80/80 [==============================] - 11s 105ms/step - loss: 0.4641 - sparse_categorical_accuracy: 0.7831 - val_loss: 0.4251 - val_sparse_categorical_accuracy: 0.8100
Epoch 2/30
80/80 [==============================] - 4s 47ms/step - loss: 0.3405 - sparse_categorical_accuracy: 0.8600 - val_loss: 0.7085 - val_sparse_categorical_accuracy: 0.7500
Epoch 3/30
80/80 [==============================] - 4s 46ms/step - loss: 0.2941 - sparse_categorical_accuracy: 0.8838 - val_loss: 0.4610 - val_sparse_categorical_accuracy: 0.8350
Epoch 4/30
80/80 [==============================] - 4s 45ms/step - loss: 0.2466 - sparse_categorical_accuracy: 0.9044 - val_loss: 0.4173 - val_sparse_categorical_accuracy: 0.8200
Epoch 5/30
80/80 [==============================] - 4s 46ms/step - loss: 0.2155 - sparse_categorical_accuracy: 0.9200 - val_loss: 0.5456 - val_sparse_categorical_accuracy: 0.8550
Epoch 6/30
80/80 [==============================] - 4s 45ms/step - loss: 0.1942 - sparse_categorical_ac

In [21]:
# How each layer got pruned

for layer in model_for_pruning.layers:
    if isinstance(layer, tf.keras.layers.Wrapper):
        weights = layer.trainable_weights
    else:
        weights = layer.weights
    for weight in weights:        
        weight_size = weight.numpy().size
        zero_num = np.count_nonzero(weight == 0)
        print(
            f'{weight.name}: {zero_num/weight_size:.2%} sparsity ',
            f'({zero_num}/{weight_size})',
        )

conv2d/kernel:0: 79.97% sparsity  (547/684)
batch_normalization/gamma:0: 0.00% sparsity  (0/76)
batch_normalization/beta:0: 0.00% sparsity  (0/76)
depthwise_conv2d/depthwise_kernel:0: 0.00% sparsity  (0/684)
conv2d_1/kernel:0: 80.00% sparsity  (4621/5776)
batch_normalization_1/gamma:0: 0.00% sparsity  (0/76)
batch_normalization_1/beta:0: 0.00% sparsity  (0/76)
depthwise_conv2d_1/depthwise_kernel:0: 0.00% sparsity  (0/684)
conv2d_2/kernel:0: 80.00% sparsity  (4621/5776)
batch_normalization_2/gamma:0: 0.00% sparsity  (0/76)
batch_normalization_2/beta:0: 0.00% sparsity  (0/76)
dense/kernel:0: 80.26% sparsity  (122/152)
dense/bias:0: 0.00% sparsity  (0/2)


# Test the model

In [22]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

10/10 [==============================] - 1s 56ms/step - loss: 0.0732 - sparse_categorical_accuracy: 0.9750


In [23]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Weight pruning with final_sparsity= {final_sparsity}')
print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

print(model_for_pruning.summary())

Weight pruning with final_sparsity= 0.8
Training Loss: 0.0404
Training Accuracy: 98.94%

Validation Loss: 0.1488
Validation Accuracy: 94.00%

Test Loss: 0.0732
Test Accuracy: 97.50%
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 30, 4, 76)        1370      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 30, 4, 76)        305       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 30, 4, 76)        1         
 PruneLowMagnitude)                                              
                                                                 
 prune

# Save the Model

### Save Keras Model

In [24]:
from time import time

timestamp = int(time())
MODEL_NAME = 'model11'
saved_model_dir = f'./saved_models/{MODEL_NAME}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/model11/assets
INFO:tensorflow:Assets written to: ./saved_models/model11/assets


### Save Hyper-Parameters and Results

In [25]:
import pandas as pd

output_dict = {
    'timestamp': timestamp,
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'test_accuracy': test_accuracy
}

df = pd.DataFrame([output_dict])

output_path='./spectrogram_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

## Convert the model if needed (could be skipped but also simply run it all)

In [26]:
!ls saved_models

1670594974  model11


In [27]:
MODEL_NAME = 'model11'

In [28]:
converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{MODEL_NAME}')
tflite_model = converter.convert()

tflite_models_dir = './tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

!ls tflite_models

model11.tflite	model11.tflite.zip
2022-12-19 17:56:06.173312: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-19 17:56:06.173352: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-19 17:56:06.174003: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/model11
2022-12-19 17:56:06.176608: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-19 17:56:06.176633: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./saved_models/model11
2022-12-19 17:56:06.182005: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-19 17:56:06.183396: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-19 17:56:06.215012: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on Saved

## Zipping the TFLite model

In [29]:
import zipfile

tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)

In [30]:
!ls tflite_models

model11.tflite	model11.tflite.zip


In [31]:
tflite_size = os.path.getsize(tflite_model_name) / 1024.0
zipped_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024.0

print(f'Original tflite size: {tflite_size:.3f} KB')
print(f'Zipped tflite size : {zipped_size:.3f} KB')

Original tflite size: 58.691 KB
Zipped tflite size : 22.340 KB


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3b2bd993-aace-454c-8357-5037007c606e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>